In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
import seaborn as sns

In [2]:
Path='/home/mgander/Atlantic/data/Viability/CCL_dict'

In [ ]:
df=pd.read_csv(f'{Path}/cellosaurus_txt.txt', sep=None)

/tmp/ipykernel_29333/3962347765.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df=pd.read_csv(f'{Path}/cellosaurus_txt.txt', sep=None)


In [70]:
len(df)/10**6

2.76701

In [71]:
# To list

In [72]:
l=[]

for i in range(len(df)):
#for i in range(1000):
    if i%10**4==0:
        print(i)
    vals=df.iloc[i].values
    vals2=[str(a) for a in vals if a!=None]
    vals2=[a for a in vals2 if a!='nan']
    string=''.join(vals2)
    
    l.append(string)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [151]:
id_indices=np.where([True if a[:2]=='ID' else False for a in l])[0]

In [217]:
ccl_ids=[]
accs=[]
syns=[]
species=[]



for i in range(len(id_indices)-1):
    ccl_ids.append(l[id_indices[i]][5:])
    
    
    corr=l[id_indices[i]:id_indices[i+1]-1]
    
    
    syn=[a[5:] for a in corr if a[:2]=='SY']
    if len(syn)>1:
        print('More than one synonims-key!!!')
    elif len(syn)==1:
        syns.append(syn[0])
    else:
        syns.append('')
        
    
    
    acc=[a[5:] for a in corr if a[:2]=='AC']
    if len(acc)>1:
        print('More than one acesion-key!!!')
        #break
    elif len(acc)==1:
        accs.append(acc[0])
    else:
        accs.append('')
        
    spe=[a[5:] for a in corr if a[:2]=='OX']
    if len(spe)>0:
        species.append(', '.join(spe))
    else:
        species.append('')

In [220]:
# The last one I add manually
ccl_ids.append('__Parent_cell_line_of_DLD1/HCT 8/HCT 15/HRT18')
accs.append('CVCL_3449')
syns.append('')
species.append('NCBI_TaxID=9606; ! Homo sapiens (Human)')

In [221]:
df0=pd.DataFrame({'Cellosaurus_ID':ccl_ids, 'Acesion':accs, 'Synonyms':syns, 'Species':species})
df0

,Cellosaurus_ID,Acesion,Synonyms,Species
0,#132 PC31SCE8,CVCL_B0T9,Z485MG70,NCBI_TaxID=10090; ! Mus musculus (Mouse)
1,#132 PL12 SCD1,CVCL_B0T8,Z485MG63,NCBI_TaxID=10090; ! Mus musculus (Mouse)
2,#15310LN,CVCL_E548,15310LN; TER461; TER461; Ter 461; TER479; TER4...,NCBI_TaxID=9606; ! Homo sapiens (Human)
3,#1615,CVCL_KA96,,"NCBI_TaxID=10090; ! Mus musculus (Mouse), NCBI..."
4,#40a,CVCL_IW91,,NCBI_TaxID=10090; ! Mus musculus (Mouse)
...,...,...,...,...
148676,ZZUSAHi001A,CVCL_ZB29,KCNA5V259IiPSC,NCBI_TaxID=9606; ! Homo sapiens (Human)
148677,ZZUSAHi002A,CVCL_ZB30,,NCBI_TaxID=9606; ! Homo sapiens (Human)
148678,ZZUSAHi003A,CVCL_A3ZF,,NCBI_TaxID=9606; ! Homo sapiens (Human)
148679,ZZUSAHi004A,CVCL_C6U7,,NCBI_TaxID=9606; ! Homo sapiens (Human)


In [249]:
# Expand synonyms

In [241]:
def purge_and_capitalize(ccl):
    ccl_str=ccl.replace('-', '')
    ccl_str=ccl_str.replace(' ', '')
    
    return(ccl_str.upper())

In [244]:
df0['Cellosaurus_ID']=[purge_and_capitalize(a) for a in df0['Cellosaurus_ID']]

In [ ]:
syns=df0['Synonyms']

In [242]:
syn_l=[]


for syn in syns:
    if ';' in syn:
        splits=syn.split('; ')
        
        ap=[]
        for s in splits:
            ap.append(purge_and_capitalize(s))
        syn_l.append(ap)
        
    else:
        syn_l.append([purge_and_capitalize(syn)])

In [258]:
df0['Synonyms']=['; '.join(a) for a in syn_l]

In [260]:
df0.to_pickle('/home/mgander/Atlantic/data/Viability/Cellosaurus_CCLs.pkl')

In [275]:
df0[df0['Acesion']=='CVCL_1774']

,Cellosaurus_ID,Acesion,Synonyms,Species
140068,TT,CVCL_1774,MTCTT,NCBI_TaxID=9606; ! Homo sapiens (Human)
